# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

* Импортируем необходимые библиотеки

In [1]:
import pandas as pd
import re

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from catboost import CatBoostClassifier

from tqdm import tqdm, tqdm_notebook, notebook
import time

import warnings
warnings.filterwarnings('ignore')

* Прочитаем предоставленные данные и сделаем первичный анализ

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')
#data = data.sample(10000)
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [4]:
data.duplicated().sum()

0

Пропусков и дубликатов в данных нет.

In [5]:
print('Доля токсичных коментариев: {0:.1%}'.format(data['toxic'].mean()))

Доля токсичных коментариев: 10.2%


Целевые классы несбалансированы.

* Обработаем текст с помощью регулярных выражений, а затем лемматизируем его

In [6]:
nltk.download('wordnet')

m = WordNetLemmatizer()

def clear_lemmatize(text):
    re_list = re.sub(r"[^a-zA-Z']", ' ', text)
    re_list = re_list.split()
    re_list = " ".join(re_list)
    lemm_list = nltk.word_tokenize(re_list)
    
    return ' '.join(m.lemmatize(i, wordnet.VERB) for i in lemm_list)

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
%%time

tqdm.pandas()
data['lemm_text'] = data['text'].progress_apply(clear_lemmatize)

100%|██████████| 159571/159571 [02:25<00:00, 1099.15it/s]

CPU times: user 2min 19s, sys: 2.63 s, total: 2min 22s
Wall time: 2min 25s


In [8]:
data.head()

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edit make under my usernam...
1,D'aww! He matches this background colour I'm s...,0,D'aww He match this background colour I 'm see...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I 'm really not try to edit war It 's ...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I ca n't make any real suggestions on imp...
4,"You, sir, are my hero. Any chance you remember...",0,You sir be my hero Any chance you remember wha...


* Разделим выборки

Разделим целевые значения и признаки

In [9]:
x = data['lemm_text']
y = data['toxic']

Выполним разделение на тестовые, валидационные и тренировочные данные

In [10]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [11]:
print('Размер тренировочной выборки- {:.0%}'.format(X_train.shape[0]/x.shape[0]))
print('Размер валидационной выборки - {:.0%}'.format(X_valid.shape[0]/x.shape[0]))
print('Размер тестовой выборки - {:.0%}'.format(X_test.shape[0]/x.shape[0]))

Размер тренировочной выборки- 80%
Размер валидационной выборки - 10%
Размер тестовой выборки - 10%


* Выполним векторизацию счетчика слов с учетом обработки стоп слов

In [12]:
stopwords = set(stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words = stopwords)
X_train = count_tf_idf.fit_transform(X_train)

Лишние символы убраны, комментарии лемматизированы, текст преобразован в векторный вид. Можно переходить к обучению моделей.

## Обучение

* DecisionTreeClassifier

In [13]:
%%time

best_model_dt = None
best_result_dt = 0
best_depth_dt=0
for depth in tqdm(range(1, 25)):
      
    model_dt = DecisionTreeClassifier(random_state=42, max_depth=depth,class_weight='balanced') 
    model_dt.fit(X_train, y_train) 
    X_valid_dt = count_tf_idf.transform(X_valid)   
    predict_dt=model_dt.predict(X_valid_dt) 
    result_dt = f1_score(y_valid, predict_dt)
    if result_dt > best_result_dt:
        best_model_dt = model_dt
        best_result_dt = result_dt 
          
        best_depth_dt = depth
f1_dt_vl = best_result_dt.round(3)       
print("F1 наилучшей модели равно:", f1_dt_vl, end='')
print(' C глубиной:',best_depth_dt)

100%|██████████| 24/24 [05:57<00:00, 14.88s/it]

F1 наилучшей модели равно: 0.622 C глубиной: 24
CPU times: user 5min 53s, sys: 281 ms, total: 5min 54s
Wall time: 5min 57s


In [14]:
X_test_DT = count_tf_idf.transform(X_test)
prediction = best_model_dt.predict(X_test_DT)
f1_dt_ts = f1_score(y_test, prediction).round(3)
print('F1 =', f1_dt_ts, '(тестовая выборка)')

F1 = 0.621 (тестовая выборка)


* LogisticRegression

In [15]:
model_LR = LogisticRegression(class_weight = 'balanced', random_state=42)
model_LR.fit(X_train, y_train)
X_valid_LR = count_tf_idf.transform(X_valid)
pred_LR = model_LR.predict(X_valid_LR)
f1 = f1_score(y_valid, pred_LR)
print('F1 =', round(f1, 3), '(валидационная выборка)')

F1 = 0.744 (валидационная выборка)


Подберем параметры модели, чтобы поднять значение метрики.

In [16]:
%%time

pipe = Pipeline([
    (
    ('model', LogisticRegression(random_state=42, solver='liblinear', max_iter=200))
    )
])

param_grid = [
    {
        'model' : [LogisticRegression(random_state=42, solver='liblinear')],
        'model__penalty' : ['l1', 'l2'],
        'model__C' : list(range(1, 15, 3))
    }
]

grid = GridSearchCV(pipe, param_grid=param_grid, scoring='f1', 
                    cv=3, verbose=True, n_jobs=-1)

best_grid = grid.fit(X_train, y_train)
print('Best parameters is:', grid.best_params_)
print('Best score is:', grid.best_score_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  3.9min finished


Best parameters is: {'model': LogisticRegression(C=4, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=42, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False), 'model__C': 4, 'model__penalty': 'l1'}
Best score is: 0.7719618425844303
CPU times: user 2min 39s, sys: 1min 19s, total: 3min 58s
Wall time: 3min 59s


In [17]:
model_LR_1 = LogisticRegression(random_state=42, C=13, penalty='l1', solver='liblinear', max_iter=200)
model_LR_1.fit(X_train, y_train)
X_valid_LR_1 = count_tf_idf.transform(X_valid)
pred_LR_1 = model_LR_1.predict(X_valid_LR_1)
f1_lr_vl = f1_score(y_valid, pred_LR_1).round(3)
print('F1 =', f1_lr_vl, '(валидационная выборка)')

F1 = 0.773 (валидационная выборка)


In [18]:
X_test_LR_1 = count_tf_idf.transform(X_test)
prediction = model_LR_1.predict(X_test_LR_1)
f1_lr_ts = f1_score(y_test, prediction).round(3)
print('F1 =', f1_lr_ts, '(тестовая выборка)')

F1 = 0.78 (тестовая выборка)


* CatBoost

In [19]:
%%time

cat_model = CatBoostClassifier(eval_metric="F1", 
                                   iterations=100, 
                                   max_depth=6, 
                                   learning_rate=0.9, 
                                   random_state=12345)
cat_model.fit(X_train, y_train, verbose=10)
X_valid_cat = count_tf_idf.transform(X_valid)
prediction = cat_model.predict(X_valid_cat)
f1_cb_vl = f1_score(y_valid, prediction).round(3)
print('F1 =', f1_cb_vl, '(валидационная выборка)')

0:	learn: 0.4943519	total: 5.75s	remaining: 9m 29s
10:	learn: 0.6911182	total: 52.6s	remaining: 7m 5s
20:	learn: 0.7389273	total: 1m 39s	remaining: 6m 12s
30:	learn: 0.7626121	total: 2m 25s	remaining: 5m 22s
40:	learn: 0.7733612	total: 3m 11s	remaining: 4m 35s
50:	learn: 0.7858495	total: 3m 58s	remaining: 3m 48s
60:	learn: 0.7933601	total: 4m 45s	remaining: 3m 2s
70:	learn: 0.7951238	total: 5m 33s	remaining: 2m 16s
80:	learn: 0.7998460	total: 6m 20s	remaining: 1m 29s
90:	learn: 0.8045693	total: 7m 7s	remaining: 42.3s
99:	learn: 0.8077267	total: 7m 50s	remaining: 0us
F1 = 0.765 (валидационная выборка)
CPU times: user 9min 11s, sys: 16 s, total: 9min 27s
Wall time: 9min 29s


In [20]:
X_test_cat = count_tf_idf.transform(X_test)
prediction = cat_model.predict(X_test_cat)
f1_cb_ts = f1_score(y_test, prediction).round(3)
print('F1 =', f1_cb_ts, '(тестовая выборка)')

F1 = 0.756 (тестовая выборка)


## Выводы

Сведём полученные результаты в одну таблицу для наглядности

In [21]:
final_tabl_index = pd.Series(['Дерево решений','Логистическая регрессия','CatBoost'])
final_tabl_data = {'F1_valid': [f1_dt_vl, f1_lr_vl, f1_cb_vl],
                   'F1_test': [f1_dt_ts, f1_lr_ts, f1_cb_ts]}

final_tabl_df = pd.DataFrame(data = final_tabl_data).set_index(final_tabl_index)
final_tabl_df

,F1_valid,F1_test
Дерево решений,0.622,0.621
Логистическая регрессия,0.773,0.780
CatBoost,0.765,0.756


Mодель LogisticRegression показала наилучший результат, набрав пороговое значение метрики $F1 = 0.78$.  
Данную модель мы можем порекомендовать магазину "ВИКИШОП" для классификации комментариев на позитивные и негативные.